In [4]:
from PIL import Image

import librosa
import matplotlib.pyplot as plt
import sounddevice as sd
import numpy as np
import math
import scipy
import os
import numpy as np


In [5]:
class FeatureExtractor:
    def __init__(self, audio, *, windowLength, overlap, sample_rate):
        self.audio = audio
        self.ffT_length = windowLength
        self.window_length = windowLength
        self.overlap = overlap
        self.sample_rate = sample_rate
        self.window = scipy.signal.hamming(self.window_length, sym=False)

    def get_stft_spectrogram(self):
        return librosa.stft(self.audio, n_fft=self.ffT_length, win_length=self.window_length, hop_length=self.overlap,
                            window=self.window, center=True)

    def get_audio_from_stft_spectrogram(self, stft_features):
        return librosa.istft(stft_features, win_length=self.window_length, hop_length=self.overlap,
                             window=self.window, center=True)

    def get_mel_spectrogram(self):
        return librosa.feature.melspectrogram(self.audio, sr=self.sample_rate, power=2.0, pad_mode='reflect',
                                           n_fft=self.ffT_length, hop_length=self.overlap, center=True)

    def get_audio_from_mel_spectrogram(self, M):
        return librosa.feature.inverse.mel_to_audio(M, sr=self.sample_rate, n_fft=self.ffT_length, hop_length=self.overlap,
                                             win_length=self.window_length, window=self.window,
                                             center=True, pad_mode='reflect', power=2.0, n_iter=32, length=None)

In [ ]:
windowLength = 256
overlap      = round(0.25 * windowLength) # overlap of 75%
ffTLength    = windowLength
inputFs      = 48e3
fs           = 16e3
numFeatures  = ffTLength//2 + 1
numSegments  = 8
print("windowLength:",windowLength)
print("overlap:",overlap)
print("ffTLength:",ffTLength)
print("inputFs:",inputFs)
print("fs:",fs)
print("numFeatures:",numFeatures)
print("numSegments:",numSegments)

In [7]:
def prepare_input_features(stft_features):
    # Phase Aware Scaling: To avoid extreme differences (more than
    # 45 degree) between the noisy and clean phase, the clean spectral magnitude was encoded as similar to [21]:
    noisySTFT = np.concatenate([stft_features[:,0:numSegments-1], stft_features], axis=1)
    stftSegments = np.zeros((numFeatures, numSegments , noisySTFT.shape[1] - numSegments + 1))

    for index in range(noisySTFT.shape[1] - numSegments + 1):
        stftSegments[:,:,index] = noisySTFT[:,index:index + numSegments]
    return stftSegments

In [6]:
file_names = [filename.split('.')[0]
              for filename in os.listdir("/home/jyothish/Projects/Audio-Denoising-with-Autoencoders/data/clean/wave")
              if filename.endswith('.wav')]

In [ ]:
for name in file_names:
    
    signal, sr = librosa.load("/home/jyothish/Projects/Audio-Denoising-with-Autoencoders/data/clean/wave/" + name + ".wav")
    
    cleanAudioFeatureExtractor = FeatureExtractor(signal, windowLength=windowLength, overlap=overlap, sample_rate=sr)
    stft_features = cleanAudioFeatureExtractor.get_stft_spectrogram()
    stft_features = np.abs(stft_features)   
    
    input_feat = prepare_input_features(stft_features)
    input_feat = np.reshape(input_feat, (input_feat.shape[0], input_feat.shape[1], 1, input_feat.shape[2]))
    input_feat = np.transpose(input_feat, (3, 0, 1, 2)).astype(np.float32)
    